In [1]:
# Import libraries
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Assign spreadsheet filename to `file`
file = 'data_sarah.xls'

# Load spreadsheet
xl = pd.ExcelFile(file)

# There are 18 sheets in total
mysheets = xl.sheet_names

description = [] # for the description of each sheet
df_raw = [] # for the df's returned by parsing the excel sheet

# Read in all descriptions and sheets
for i, sheet in enumerate(mysheets):

    # The top left cell of each sheet contains a description that puts the 
    # data in the sheet into context
    description.append(xl.parse(sheet, header=None, nrows=1)[0][0])

    # The actual data table starts in row 3
    df_raw.append(xl.parse(sheet, header = 2))
    
# Print one as an example
df_raw[5]

,Theme,Question,most deprived 1,deprived-2,deprived-3,deprived-4,COL1,COL2,COL3,COL4
0,Access,If you ask to make an appointment with a docto...,0.853,0.849,0.930,0.974,-,-,-,N
1,Access,NaN,"(0.800 ,0.909)","(0.799 ,0.902)","(0.875 ,0.988)","(0.917 ,1.033)",-,-,-,N
2,Referrals,Thinking about the last time your GP referred ...,1.240,1.193,1.102,1.082,+,+,+,+
3,Referrals,NaN,"(1.151 ,1.335)","(1.113 ,1.279)","(1.030 ,1.179)","(1.012 ,1.156)",+,+,+,+
4,At your GP surgery,How clean is your GP surgery or health centre?,1.572,1.307,1.214,1.072,+,+,+,+
5,At your GP surgery,NaN,"(1.487 ,1.661)","(1.242 ,1.375)","(1.155 ,1.275)","(1.022 ,1.123)",+,+,+,+
6,Doctor,I felt that the doctor had all the information...,1.135,1.109,1.043,0.995,+,+,N,N
7,Doctor,NaN,"(1.049 ,1.229)","(1.029 ,1.195)","(0.972 ,1.120)","(0.929 ,1.067)",+,+,N,N
8,Doctor,I felt confident in the doctor's ability to tr...,1.048,1.029,1.018,0.941,N,N,N,N
9,Doctor,NaN,"(0.968 ,1.134)","(0.955 ,1.108)","(0.948 ,1.093)","(0.878 ,1.008)",N,N,N,N


In [3]:
df = [] # clean dataframes will go in here later

# iterate through all parsed sheets and clean/reformat them
for dfr in df_raw:
    
    # Remove COLx (they only describe information that is already contained 
    # in the other columns)
    cols = dfr.columns
    pattern = r'^COL*'
    cols_to_keep = cols[~cols.str.match(pattern)]
    dfr = dfr[cols_to_keep]
    
    # Because of how the Excel sheet was formatted, we need to split out the
    # odd rows
    
    # Create index lists for odd and even rows
    even = np.array(range(0, len(dfr), 2))
    odd = np.array(range(1, len(dfr), 2))
    
    # Separate dataframes - we will add the odd rows back into the df with the
    # even rows later
    df_clean = dfr.iloc[even, :].reset_index(drop=True) # even rows
    df_odd = dfr.iloc[odd, 2:].reset_index(drop=True) # odd rows
    
    # Melt dataframe into long shape
    df_clean = df_clean.melt(id_vars=['Theme', 'Question'], 
                 var_name='group', 
                 value_name='odds_ratio')
    
    # Theme and question are mostly missing because of the merged cells in the
    # Excel sheet, but they are ordered correctly, so I can just replace those
    # columns with the ones from the dataframe with the even rows
    df_odd['Theme'] = df_clean['Theme']
    df_odd['Question'] = df_clean['Question']
    
    # Melt the dataframe with the odd rows as well
    df_odd = df_odd.melt(id_vars=['Theme', 'Question'], 
                 var_name='group', 
                 value_name='conf')
    
    # Split the column with the confidence interval into separate columns
    conf_split = df_odd['conf'].str.replace(r'\(|\)', '').str.split(',', expand=True)
    # Add into the clean dataframe
    df_clean['conf_lower'] = conf_split[0]
    df_clean['conf_upper'] = conf_split[1]
    
    # lowercase column names
    df_clean.columns = df_clean.columns.str.lower()
        
    # convert columns with numbers to numeric type
    for colname in df_clean.columns[3:]:
        df_clean[colname] = pd.to_numeric(df_clean[colname])

    # Append clean dataframe to list
    df.append(df_clean)

In [4]:
description


['Odds ratios by gender with females as the base group',
 'Odds ratios by age, interpreted as the odds for a patient one year older than the mean age (of surveyed patients) compared to a patient who has the mean age. Note: The effect may seem quite small, but it must be remembered that the difference is for a one year increase in age and the survey covered a large age range.',
 'Odds ratios by ethnicity with those identifying as white as the base group',
 'Odds ratios by religion with those identifying as belonging to Church of Scotland as the base group',
 'Odds ratios by sexual orientation with those identifying as heterosexual / straight as the base group',
 'Odds ratios by deprivation (based on SIMD) with those in the least deprived quintile (Quntile 5) as the base group',
 "Odds ratios by urban-rural classification (based on SG's 6-fold UR Classification) with those living in large urban areas as the base group",
 'Odds ratios by work status with those who are retired as the base 

In [5]:
# Add additional information from the descriptions, then merge

info = [["Gender", "Female"],
        ["Age", "Mean age"],
        ["Ethnicity", "White"],
        ["Religion",  "Church of Scotland"],
        ["Sexual orientation", "Heterosexual"],
        ["Deprivation (SIMD)", "Quintile 5"],
        ["Urban-Rural", "Large urban areas"],
        ["Work Status", "Retired"],
        ["Carers", "Not a carer"],
        ["Health status", "Good"],
        ["Limitation of day-to-day activities", "Not limited"],
        ["Interpretation, translation or other communication support needs", "No"],
        ["Long-term condition", "None"],
        ["GP Practice Size", "2500 - 4999"],
        ["Percentage of GP practice list in 15% of most deprived areas", "Mean"],
        ["Frequency of contact with GP Practice", "2-4 times"],
        ["First OOH service contacted", "NHS24"],
        ["First OOH service treated or advised by", "Phone advice only"]]

# Insert two additional columns for the additional information
for i in range(len(df)):
    df[i].insert(2, 'characteristic', info[i][0])
    df[i].insert(3, 'base_group', info[i][1])

# Concatenate all dataframes, yielding one big dataframe df which contains all our data
df = pd.concat(df)

# Reset index to number rows sequentially
df = df.reset_index(drop=True)

In [6]:
df

,theme,question,characteristic,base_group,group,odds_ratio,conf_lower,conf_upper
0,Access,Thinking of the last time you contacted this G...,Gender,Female,Male,1.064,1.026,1.104
1,Access,The last time you needed to see or speak to a ...,Gender,Female,Male,0.774,0.742,0.808
2,Access,If you ask to make an appointment with a docto...,Gender,Female,Male,1.179,1.137,1.222
3,Access,When you arrange to see a doctor at your GP su...,Gender,Female,Male,1.240,1.195,1.286
4,Access,Overall how would you rate the arrangements fo...,Gender,Female,Male,1.180,1.144,1.216
5,Access,Overall how would you rate the arrangements fo...,Gender,Female,Male,1.223,1.174,1.274
6,Referrals,Thinking about the last time your GP referred ...,Gender,Female,Male,1.053,1.009,1.100
7,At your GP surgery,How helpful do you find the receptionists at y...,Gender,Female,Male,1.076,1.015,1.142
8,At your GP surgery,How do you feel about how long you usually hav...,Gender,Female,Male,1.182,1.136,1.230
9,Doctor,The doctor listened to me,Gender,Female,Male,1.192,1.120,1.269


In [7]:
# df.to_csv('clean_data.csv')

In [14]:
df.groupby(['characteristic', 'group', 'base_group']).mean()#.to_csv('by_group.csv')

odds_ratio  \
characteristic                                     group                                     base_group                       
Age                                                Age                                       Mean age              1.015538   
Carers                                             Yes, 20-34 \nhours a week                 Not a carer           0.835176   
                                                   Yes, 35 or more hours a week              Not a carer           0.897206   
                                                   Yes, up to 19 hours a week                Not a carer           0.811088   
Deprivation (SIMD)                                 deprived-2                                Quintile 5            1.118333   
                                                   deprived-3                                Quintile 5            1.056400   
                                                   deprived-4                                Quintile 5            1.043933   
                                                   most deprived 1                           Quintile 5            1.188467   
Ethnicity                                          African, Caribbean or Black               White                 0.891842   
                                                   Asian                                     White                 0.713316   
                                                   Mixed                                     White                 0.946158   
                                                   Other                                     White                 0.645474   
First OOH service contacted                        999 Emergency service                     NHS24                 1.338167   
                                                   A&E / Casualty                            NHS24                 1.122667   
                                                   District /Ccommunity Nurse                NHS24                 1.928333   
                                                   My own GP surgery                         NHS24                 1.328167   
                                                   Other                                     NHS24                 1.270833   
                                                   Out of Hours Service                      NHS24                 1.108000   
                                                   Pharmacist / Chemist                      NHS24                 1.344167   
First OOH service treated or advised by            A&E / \nCasualty                          Phone advice only     1.321000   
                                                   Emergency dental                          Phone advice only     1.898667   
                                                   Home visit \nfrom another \ndoctor/ nurse Phone advice only     1.534500   
                                                   My own GP surgery                         Phone advice only     2.151500   
                                                   Other                                     Phone advice only     0.819833   
                                                   Out of Hours Service                      Phone advice only     1.679833   
                                                   Paramedics                                Phone advice only     2.960333   
                                                   Pharmacist / Chemist                      Phone advice only     2.353000   
Frequency of contact with GP Practice              5-10 times                                2-4 times             1.046960   
                                                   Can't remember / don't know               2-4 times             0.874360   
                                                   More than 10 times                        2-4 times             1.151720   
...                                              

In [13]:
df.characteristic.unique()

array(['Gender', 'Age', 'Ethnicity', 'Religion', 'Sexual orientation',
       'Deprivation (SIMD)', 'Urban-Rural', 'Work Status', 'Carers',
       'Health status', 'Limitation of day-to-day activities',
       'Interpretation, translation or other communication support needs',
       'Long-term condition', 'GP Practice Size',
       'Percentage of GP practice list in 15% of most deprived areas',
       'Frequency of contact with GP Practice',
       'First OOH service contacted',
       'First OOH service treated or advised by'], dtype=object)